# Portfolio manager

In [1]:
# run only once.
import os
os.chdir("..")
!pwd
!pyenv version
%load_ext autoreload
%autoreload 2

/Users/loicmorel/code/projetcs/crypto_assistant
crypto_assistant (set by PYENV_VERSION environment variable)


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from app.data_mgt.datamgt import ohlcv_from_csv_to_df
from datetime import timedelta, date
import os
# from app.portfolio_manager.portfolio import Portfolio
from app.utils import daterange
from app.modeling.predict import RNN_model_prediction
from app.modeling.predict import CNN_model_prediction
from app.modeling.utils import adding_target_to_data
from app.config import Config
import ast
import random

## Class portfolio

In [33]:
class Portfolio():
    def __init__(self, name, init_assets, coins):
        self.name = name
        self.init_assets = init_assets
        self.coins = coins
        self.data_folder = Config().project.get('bucket_data_folder')
        self.portfolio = pd.DataFrame(columns=[
            'date',
            't_bh_alloc',
            't_bh_perf',
            't_bh_val',
            't_ai_alloc',
            't_ai_stable',
            't_ai_perf',
            't_ai_val'
        ], dtype=object)

        self.data_1h = {}
        self.data_1d = {}
        self.metadata = {}
        self.status = 'init'

        # for all coins in the coins dict
        for coin  in self.coins:

            # collecting data for the coin.
            if self._set_ohlcv_data(coin, '1d') == -1:
                self.status = f'no data found for {coin} @1d'
                return self.status

            if self._set_ohlcv_data(coin, '1h') == -1:
                self.status = f'no data found for {coin} @1h'
                return self.status

    def delete_protfolio(self):
        self.__init__


    def get_coins(self):
        return self.coins

    def set_allocation(self, coins_dict, date, rand_pred):

        add_row = {}
        # add row with current date
        add_row['date'] = date
        total_bh_alloc = total_bh_perf = total_bh_val = 0
        total_ai_alloc = total_ai_perf = total_ai_val = 0

        # colelct previous valuations
        if len(self.portfolio) > 1:
            p_t_bh_val = self.portfolio['t_bh_val'].iloc[-1]
            p_t_ai_val = self.portfolio['t_ai_val'].iloc[-1]
        else:
            p_t_bh_val = p_t_ai_val = self.init_assets

        # generate prediction for all coins ()
        ai_alloc_d = self._get_ai_alloc(coins_dict, date, rand_pred)
        if ai_alloc_d == -1:
            self.status = f'error prediction (data or model)'
            return self.status

        # for all coins in the coins dict
        for coin, alloc in coins_dict.items():

            # check if date in data
            if not self._check_date_in_data(coin, date):
                self.status = f'no date found in the dataset ohlcv for {coin}'
                return self.status

            # calculate performance
            perf = self._perf_calc(self.data_1d[coin], date)

            # calculate valuation
            val = self._valuation_calc(self.data_1d[coin], date)

            # add rows for buy and hold
            add_row[f'{coin}_bh_alloc'] = alloc
            add_row[f'{coin}_perf'] = perf
            add_row[f'{coin}_val'] = val
            add_row[f'{coin}_bh_nb'] = (p_t_bh_val * (alloc/100)) / val
            total_bh_alloc += alloc
            total_bh_perf += perf * (alloc/100)
            total_bh_val += (p_t_bh_val * (alloc/100))

            # add rows for predictions
            ai_alloc = ai_alloc_d.get(coin)
            add_row[f'{coin}_ai_alloc'] = ai_alloc
            add_row[f'{coin}_bh_nb'] = (p_t_bh_val * (ai_alloc/100)) / val
            total_ai_perf += perf * (ai_alloc/100)
            total_ai_val += (p_t_ai_val * (ai_alloc/100))

        # add totals for buy and hold
        add_row['t_bh_alloc'] = total_bh_alloc
        add_row['t_bh_perf'] = total_bh_perf
        add_row['t_bh_val'] = p_t_bh_val + (p_t_bh_val * total_bh_perf/100)

        # add totals for prediction
        add_row['t_ai_alloc'] = sum(ai_alloc_d.values())
        add_row['t_ai_perf'] = total_ai_perf
        add_row['t_ai_stable'] = 100 - add_row['t_ai_alloc']
        add_row['t_ai_val'] = p_t_ai_val + (p_t_ai_val * total_ai_perf/100 + add_row['t_ai_stable']/100)
        

        self.portfolio = pd.concat([self.portfolio, pd.DataFrame([add_row])], axis=0, ignore_index=True)
        self.portfolio = self.portfolio.fillna(0)
        self.status = 'ready'
        return self.status


    def _set_ohlcv_data(self, coin, freq):
        # list avalable data of frequency
        for f in os.listdir(self.data_folder):
            if f'{coin}USDT_{freq}' in f:
                data = ohlcv_from_csv_to_df(f'{self.data_folder}/{f}')

                if freq == '1d':
                    self.data_1d[coin] = data

                elif freq == '1h':
                    self.data_1h[coin] = data

                list_of_metadata = f.split('_')
                self.metadata[coin] = {
                    'symbol' : list_of_metadata[1],
                    'frequency' : list_of_metadata[2],
                    'start_ts' : list_of_metadata[4],
                    'end_ts' : list_of_metadata[6].rsplit( ".", 1 )[0]
                }
                return 0
        return -1

    def _perf_calc(self, data, date):
        p = data['Close'][(data['o_ts'] == date)].values[0]
        p_1 = data['Close'][(data['o_ts'] == (date - timedelta(days=1)))].values[0]
        return (p - p_1) / p_1 * 100

    def _valuation_calc(self, data, date):
        return (data['Close'][(data['o_ts'] == date)].values[0] +
                data['Open'][(data['o_ts'] == date)].values[0]) / 2

    def _check_date_in_data(self, coin, date):
        return len(self.data_1d[coin][self.data_1d[coin]['o_ts'] == date])

    def _get_ai_alloc(self, coins_dict, date, random):

        pred = {}

        if random == 0:
            #print('manage prediction and new allocation for:', coins_dict, date)

            total_value = 0

            for coin, alloc in coins_dict.items():
                #print('collect prediction for:', coin, date)

                metadata = self.metadata[coin]

                #y_pred = RNN_model_prediction(metadata, self.data_1h[coin], date)
                y_pred = CNN_model_prediction(metadata, self.data_1h[coin], date)

                #print('prediction:', metadata.get('symbol'), date, y_pred)

                if y_pred == -1:
                    return -1

                alloc_dict = {
                    'NP': +0,
                    'HH': -40,
                    'LL': +60,
                    'LH': -20,
                    'HL': +20
                }
                pred[coin] = coins_dict.get(coin) + alloc_dict.get(y_pred)
                total_value += pred[coin]

            to_reallocate = 100 - total_value
            
            #print('to_reallocate:', to_reallocate)

#             to_reallocate
            total_coins_alloc = 0
            
            if to_reallocate < 0:
                # we squize to 100%
                for coin, alloc in coins_dict.items():
                    pred[coin] = pred[coin] + to_reallocate * alloc/100
                    total_coins_alloc += pred[coin]
            
            #print('total_coins_alloc:', total_coins_alloc)

        else:
            rand_alloc = self._generate_random_values(len(coins_dict), 100)

            i = 0
            for coin, alloc in coins_dict.items():
                pred[coin] = np.array(rand_alloc).tolist()[i]
                i = i + 1

        return pred

    def _generate_random_values(self, nb_val, cap):
        ran_val = []
        for i in range(nb_val-1):
            ran_val.append(random.randrange(0, int(cap/3)))
        arr = np.array(ran_val)
        final_value = 100 - arr.sum()
        arr = np.append(arr, final_value)
        return arr


## Create a portfolio

In [49]:
#### API ####

# define the name
pf_name = 'Loic'

# define initial assets (USD)
ps_init_assets = 1000

# today
date = datetime.strptime("2022-05-02_00:00:00", "%Y-%m-%d_%H:%M:%S")

# define allocation dictionary
coins_dict = {
    'ETH': 20,
    'VITE': 30,
    'BTC': 50
}

#############

pf = Portfolio(pf_name, ps_init_assets, list(coins_dict.keys()))

# 0 = model allocation, 1 = random allocation
rand_pred = 0
        
# generate the portfolio for the analysis period
for date in daterange(date - timedelta(days=20), date):
    pf.set_allocation(coins_dict, date, rand_pred)
    
bh_perf = (pf.portfolio['t_bh_val'].iloc[-1] - ps_init_assets)/ps_init_assets * 100
ai_perf = (pf.portfolio['t_ai_val'].iloc[-1] - ps_init_assets)/ps_init_assets * 100
    
print('b&h performance:', round(bh_perf, 2), '%')
print('ai performance:', round(ai_perf, 2), '%')
print('ai(%) profits/loss:', round((ai_perf - bh_perf), 2), '%')

pf.portfolio

b&h performance: -9.5 %
ai performance: -8.74 %
ai(%) profits/loss: 0.76 %


,date,t_bh_alloc,t_bh_perf,t_bh_val,t_ai_alloc,t_ai_stable,t_ai_perf,t_ai_val,ETH_bh_alloc,ETH_perf,...,VITE_bh_alloc,VITE_perf,VITE_val,VITE_bh_nb,VITE_ai_alloc,BTC_bh_alloc,BTC_perf,BTC_val,BTC_bh_nb,BTC_ai_alloc
0,2022-04-12,100,0.928767,1009.287665,100.0,0.0,1.197943,1011.979429,20.0,1.608194,...,30.0,-0.271897,0.051460,2331.908278,12.0,50.0,1.377394,39802.695,0.020099,80.0
1,2022-04-13,100,2.842719,1028.427190,100.0,0.0,2.978258,1029.782582,20.0,2.993071,...,30.0,3.018500,0.052125,12661.870504,66.0,50.0,2.677109,40611.370,0.002462,10.0
2,2022-04-14,100,-3.517163,992.255727,60.0,40.0,-2.345377,1006.030295,20.0,-3.088241,...,30.0,-4.782609,0.051625,5976.332338,30.0,50.0,-2.929465,40545.080,0.002537,10.0
3,2022-04-15,100,1.648323,1008.611311,100.0,0.0,1.648323,1022.612928,20.0,0.614829,...,30.0,2.541195,0.051005,5836.226217,30.0,50.0,1.525998,40247.135,0.012327,50.0
4,2022-04-16,100,-0.340032,1005.181705,100.0,0.0,-0.340032,1019.135713,20.0,0.616334,...,30.0,-0.832527,0.051385,5888.554893,30.0,50.0,-0.427082,40465.305,0.012463,50.0
5,2022-04-17,100,-2.410955,980.947222,40.0,60.0,-0.998467,1009.559980,20.0,-2.328637,...,30.0,-3.592347,0.050300,1998.373172,10.0,50.0,-1.735048,40028.410,0.002511,10.0
6,2022-04-18,100,1.775818,998.367063,80.0,20.0,1.324020,1023.126758,20.0,2.258991,...,30.0,-0.303767,0.049275,5972.281413,30.0,50.0,2.830300,40239.620,0.012189,50.0
7,2022-04-19,100,2.186496,1020.196316,60.0,40.0,1.508034,1038.955858,20.0,1.512325,...,30.0,3.453179,0.050085,5980.036316,30.0,50.0,1.696154,41147.155,0.002426,10.0
8,2022-04-20,100,0.448967,1024.776663,40.0,60.0,-0.522684,1034.125405,20.0,-0.829526,...,30.0,2.591793,0.051590,-1977.507881,-10.0,50.0,-0.325331,41425.690,0.007388,30.0
9,2022-04-21,100,-3.161403,992.379341,100.0,0.0,-3.161403,1001.432532,20.0,-3.005813,...,30.0,-4.995215,0.050965,6032.237789,30.0,50.0,-2.123352,40919.100,0.012522,50.0


### intresting code

In [8]:
df['ctime']  = pd.to_datetime(df['ctime'], unit='s')
df           = df.set_index('ctime')
df.resample('1H',  how='ohlc', axis=0, fill_method='bfill')

https://stackoverflow.com/questions/9944436/converting-ohlc-stock-data-into-a-different-timeframe-with-python-and-pandas?rq=1

SyntaxError: invalid syntax (2609432285.py, line 5)